# 2. Preprocess the variables

**Outline:**
- Preprocess Activity and CumTimeInterval columns
- Get padding mask
- Get available/nan mask

In [1]:
import os
import sys
import argparse
import pandas as pd
import numpy as np
import pickle

In [2]:
from dateutil.parser import parse
from datetime import datetime
import time
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

pd.options.mode.chained_assignment = None #to run loop quicker without warnings

In [3]:
name = 'small_log'

args = {
    'data_dir': '../data/',
    'data_file': name + '.csv',
    'input_dir': '../input/{}/'.format(name),  
    'nan_pct': 0.3,
    'train_pct': 0.6,
    'val_pct': 0.2,
}

args = argparse.Namespace(**args)

In [4]:
file_name = os.path.join(args.input_dir, 'parameters_{}.pkl'.format(args.nan_pct))

with open(file_name, 'rb') as f:
    most_frequent_activity = pickle.load(f)
    first_timestamp = pickle.load(f)
    avai_instance = pickle.load(f)
    nan_instance = pickle.load(f)
    train_size = pickle.load(f)
    val_size = pickle.load(f)
    test_size = pickle.load(f)
    train_row_num = pickle.load(f)
    val_row_num = pickle.load(f)
    test_row_num = pickle.load(f)

In [5]:
sys.path.insert(0, './../utils/')
from utils import *

## Load data

In [6]:
df_name = os.path.join(args.input_dir, 'complete_df_full_{}.csv'.format(args.nan_pct))
df = pd.read_csv(df_name)

missing_df_name = os.path.join(args.input_dir, 'missing_df_full_{}.csv'.format(args.nan_pct))
missing_df = pd.read_csv(missing_df_name)

## Preprocess data


- Normalize CumTimeInterval using minmaxScaler for each case.
- One hot encode activity
- Get available/nan mask
- Replace nan with 0
- Pad with 0 to get same size cases

In [7]:
df.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval
0,1,Activity A,1970-01-01 09:00:00,0.0
1,1,Activity B,1970-01-01 10:00:00,3600.0
2,1,Activity C,1970-01-01 11:00:00,7200.0
3,1,Activity D,1970-01-01 12:00:00,10800.0
4,1,Activity E,1970-01-01 13:00:00,14400.0


In [8]:
len(df['Activity'].unique())

14

In [9]:
missing_df.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval
0,1,Activity A,1970-01-01 09:00:00,0.0
1,1,NaN,1970-01-01 10:00:00,3600.0
2,1,Activity C,1970-01-01 11:00:00,7200.0
3,1,NaN,NaN,NaN
4,1,NaN,1970-01-01 13:00:00,14400.0


### 1. Normalize

In [10]:
groupByCase = df.groupby(['CaseID'])
missing_groupByCase = missing_df.groupby(['CaseID'])

In [16]:
normalized_complete_df = pd.DataFrame(columns=list(df)+['NormalizedTime'])
normalized_missing_df = pd.DataFrame(columns=list(df)+['NormalizedTime'])
min_max_storage = {}

for i, j in zip(groupByCase, missing_groupByCase):
    temp, missing_temp, missing_case_storage = minmaxScaler(i[0], i[1], j[1])
    normalized_complete_df = normalized_complete_df.append(temp)
    normalized_missing_df = normalized_missing_df.append(missing_temp)
    min_max_storage.update(missing_case_storage)

In [17]:
len(min_max_storage), len(groupByCase), len(missing_groupByCase)

(2000, 2000, 2000)

In [18]:
normalized_complete_df.shape

(28000, 5)

In [19]:
normalized_missing_df.shape

(28000, 5)

In [20]:
normalized_complete_df_name = os.path.join(args.input_dir, 'normalized_complete_df_{}.csv'.format(args.nan_pct))
normalized_complete_df.to_csv(normalized_complete_df_name, index=False)

normalized_missing_df_name = os.path.join(args.input_dir, 'normalized_missing_df_{}.csv'.format(args.nan_pct))
normalized_missing_df.to_csv(normalized_missing_df_name, index=False)

### 2. One hot encode

- One hot encode for all categorical variables
- All columns are 0 for nan value

In [21]:
cat_var = ['Activity']

In [22]:
# OneHotEncoding: get k dummies out of k categorical levels (drop_first=False)
enc_complete_df = OHE(normalized_complete_df, cat_var)
enc_missing_df = OHE(normalized_missing_df, cat_var)

In [23]:
enc_complete_df.head()

,CaseID,CompleteTimestamp,CumTimeInterval,NormalizedTime,Activity_Activity A,Activity_Activity B,Activity_Activity C,Activity_Activity D,Activity_Activity E,Activity_Activity F,Activity_Activity G,Activity_Activity H,Activity_Activity I,Activity_Activity J,Activity_Activity K,Activity_Activity L,Activity_Activity M,Activity_Activity N
0,1,1970-01-01 09:00:00,0.0,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1970-01-01 10:00:00,3600.0,0.125000,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1970-01-01 11:00:00,7200.0,0.249999,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,1,1970-01-01 12:00:00,10800.0,0.374999,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,1,1970-01-01 13:00:00,14400.0,0.499998,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [24]:
enc_missing_df.head()

,CaseID,CompleteTimestamp,CumTimeInterval,NormalizedTime,Activity_Activity A,Activity_Activity B,Activity_Activity C,Activity_Activity D,Activity_Activity E,Activity_Activity F,Activity_Activity G,Activity_Activity H,Activity_Activity I,Activity_Activity J,Activity_Activity K,Activity_Activity L,Activity_Activity M,Activity_Activity N
0,1,1970-01-01 09:00:00,0.0,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1970-01-01 10:00:00,3600.0,0.142857,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1970-01-01 11:00:00,7200.0,0.285713,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,1,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1970-01-01 13:00:00,14400.0,0.571426,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 3. Get masks

**Note:**
- **nan_index_df**: 1: missing element, 0: available element 
- **avai_index_df**: 1: available element, 0: missing element
- **pad_index_df**: 1: all element (prepare for padding later)

### Mask for nan and avai

In [25]:
c_df = enc_complete_df.copy()
m_df = enc_missing_df.copy()
enc_complete_df_w_normalized_time = c_df.drop(['CompleteTimestamp', 'CumTimeInterval'], axis=1)
enc_missing_df_w_normalized_time = m_df.drop(['CompleteTimestamp', 'CumTimeInterval'], axis=1)

In [26]:
c_df = enc_complete_df.copy()
m_df = enc_missing_df.copy()
enc_complete_df_w_time = c_df.drop(['CompleteTimestamp', 'NormalizedTime'], axis=1)
enc_missing_df_w_time = m_df.drop(['CompleteTimestamp', 'NormalizedTime'], axis=1)

In [27]:
avai_index_df = enc_missing_df_w_time.copy()
nan_index_df = enc_missing_df_w_time.copy()

In [28]:
# mask for Time
for row in range(enc_missing_df_w_time.shape[0]):
    if np.isnan(enc_missing_df_w_time.loc[row, 'CumTimeInterval']): # if nan Time
        avai_index_df.loc[row, 'CumTimeInterval'] = 0
        nan_index_df.loc[row, 'CumTimeInterval'] = 1
    else:
        avai_index_df.loc[row, 'CumTimeInterval'] = 1
        nan_index_df.loc[row, 'CumTimeInterval'] = 0

In [29]:
nan_index_df.head()

,CaseID,CumTimeInterval,Activity_Activity A,Activity_Activity B,Activity_Activity C,Activity_Activity D,Activity_Activity E,Activity_Activity F,Activity_Activity G,Activity_Activity H,Activity_Activity I,Activity_Activity J,Activity_Activity K,Activity_Activity L,Activity_Activity M,Activity_Activity N
0,1,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
# mask for Activity
for row in range(enc_missing_df_w_time.shape[0]):
    if np.any(enc_missing_df_w_time.iloc[row,2:]>0): #if avai Time
        avai_index_df.iloc[row, 2:] = 1
        nan_index_df.iloc[row, 2:] = 0
    else:
        avai_index_df.iloc[row, 2:] = 0
        nan_index_df.iloc[row, 2:] = 1

In [31]:
nan_index_df.head()

,CaseID,CumTimeInterval,Activity_Activity A,Activity_Activity B,Activity_Activity C,Activity_Activity D,Activity_Activity E,Activity_Activity F,Activity_Activity G,Activity_Activity H,Activity_Activity I,Activity_Activity J,Activity_Activity K,Activity_Activity L,Activity_Activity M,Activity_Activity N
0,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1


### Mask for 0-padding

In [32]:
pad_index_df = enc_complete_df.copy()
cols = [x for x in list(pad_index_df) if x != 'CaseID']
pad_index_df.loc[:, cols] = 1
#padding_df[cols] = 1

In [34]:
pad_index_df.head(10)

,CaseID,CompleteTimestamp,CumTimeInterval,NormalizedTime,Activity_Activity A,Activity_Activity B,Activity_Activity C,Activity_Activity D,Activity_Activity E,Activity_Activity F,Activity_Activity G,Activity_Activity H,Activity_Activity I,Activity_Activity J,Activity_Activity K,Activity_Activity L,Activity_Activity M,Activity_Activity N
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
8,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


## Replace nan with 0

In [35]:
enc_missing_df_w_normalized_time.head()

,CaseID,NormalizedTime,Activity_Activity A,Activity_Activity B,Activity_Activity C,Activity_Activity D,Activity_Activity E,Activity_Activity F,Activity_Activity G,Activity_Activity H,Activity_Activity I,Activity_Activity J,Activity_Activity K,Activity_Activity L,Activity_Activity M,Activity_Activity N
0,1,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0.142857,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0.285713,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,1,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0.571426,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
enc_missing_df_w_normalized_time.fillna(0, inplace=True)
enc_missing_df_w_time.fillna(0, inplace=True)

In [37]:
enc_missing_df_w_normalized_time.head()

,CaseID,NormalizedTime,Activity_Activity A,Activity_Activity B,Activity_Activity C,Activity_Activity D,Activity_Activity E,Activity_Activity F,Activity_Activity G,Activity_Activity H,Activity_Activity I,Activity_Activity J,Activity_Activity K,Activity_Activity L,Activity_Activity M,Activity_Activity N
0,1,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0.142857,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0.285713,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0.571426,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Pad with 0

Eliminate 'CaseID'

**To do:**
- Find longest case
- Vectorize based on CaseID to get input

In [38]:
enc_complete_w_normalized_time_groupByCase = enc_complete_df_w_normalized_time.groupby(['CaseID'])
enc_missing_w_normalized_time_groupByCase = enc_missing_df_w_normalized_time.groupby(['CaseID'])

enc_complete_w_time_groupByCase = enc_complete_df_w_time.groupby(['CaseID'])
enc_missing_w_time_groupByCase = enc_missing_df_w_time.groupby(['CaseID'])

avai_index_df_groupByCase = avai_index_df.groupby(['CaseID'])
nan_index_df_groupByCase = nan_index_df.groupby(['CaseID'])
pad_index_df_groupByCase = pad_index_df.groupby(['CaseID'])

In [39]:
maxlen = findLongestLength(groupByCase)
print('Length of longest case: {}'.format(maxlen))

Length of longest case: 14


In [40]:
cols_w_time = [i for i in list(enc_complete_df_w_time) if i != 'CaseID']
cols_w_time

['CumTimeInterval',
 'Activity_Activity A',
 'Activity_Activity B',
 'Activity_Activity C',
 'Activity_Activity D',
 'Activity_Activity E',
 'Activity_Activity F',
 'Activity_Activity G',
 'Activity_Activity H',
 'Activity_Activity I',
 'Activity_Activity J',
 'Activity_Activity K',
 'Activity_Activity L',
 'Activity_Activity M',
 'Activity_Activity N']

In [41]:
cols_w_normalized_time = [i for i in list(enc_complete_df_w_normalized_time) if i != 'CaseID']
cols_w_normalized_time

['NormalizedTime',
 'Activity_Activity A',
 'Activity_Activity B',
 'Activity_Activity C',
 'Activity_Activity D',
 'Activity_Activity E',
 'Activity_Activity F',
 'Activity_Activity G',
 'Activity_Activity H',
 'Activity_Activity I',
 'Activity_Activity J',
 'Activity_Activity K',
 'Activity_Activity L',
 'Activity_Activity M',
 'Activity_Activity N']

In [45]:
def getInput(groupByCase, cols, maxlen):
    full_list = []
    for case, data in groupByCase:
        data = data[cols]
        temp = data.to_numpy()
        temp = padwithzeros(temp, maxlen)
        full_list.append(temp)
    inp = np.array(full_list)
    return inp

In [46]:
vectorized_complete_df_w_normalized_time = getInput(enc_complete_w_normalized_time_groupByCase, cols_w_normalized_time, maxlen)
vectorized_missing_df_w_normalized_time = getInput(enc_missing_w_normalized_time_groupByCase, cols_w_normalized_time, maxlen)

vectorized_complete_df_w_time = getInput(enc_complete_w_time_groupByCase, cols_w_time, maxlen)
vectorized_missing_df_w_time = getInput(enc_missing_w_time_groupByCase, cols_w_time, maxlen)

vectorized_avai_index_df = getInput(avai_index_df_groupByCase, cols_w_time, maxlen)
vectorized_nan_index_df = getInput(nan_index_df_groupByCase, cols_w_time, maxlen)
vectorized_pad_index_df = getInput(pad_index_df_groupByCase, cols_w_time, maxlen)

In [47]:
vectorized_complete_df_w_normalized_time.shape, vectorized_missing_df_w_normalized_time.shape

((2000, 14, 15), (2000, 14, 15))

In [48]:
vectorized_complete_df_w_time.shape, vectorized_missing_df_w_time.shape

((2000, 14, 15), (2000, 14, 15))

In [49]:
vectorized_avai_index_df.shape, vectorized_nan_index_df.shape, vectorized_pad_index_df.shape

((2000, 14, 15), (2000, 14, 15), (2000, 14, 15))

# Split and save data

In [50]:
# Splitting: 70% train, 10% validation, 20% test

#train
complete_matrix_w_normalized_time_train = vectorized_complete_df_w_normalized_time[:train_size]
missing_matrix_w_normalized_time_train = vectorized_missing_df_w_normalized_time[:train_size]

avai_matrix_train = vectorized_avai_index_df[:train_size]
nan_matrix_train = vectorized_nan_index_df[:train_size]

#validate
complete_matrix_w_normalized_time_val = vectorized_complete_df_w_normalized_time[train_size:train_size+val_size]
missing_matrix_w_normalized_time_val = vectorized_missing_df_w_normalized_time[train_size:train_size+val_size]

avai_matrix_val = vectorized_avai_index_df[train_size:train_size+val_size]
nan_matrix_val = vectorized_nan_index_df[train_size:train_size+val_size]
pad_matrix_val = vectorized_pad_index_df[train_size:train_size+val_size]

#test
complete_matrix_w_normalized_time_test = vectorized_complete_df_w_normalized_time[train_size+val_size:]
missing_matrix_w_normalized_time_test = vectorized_missing_df_w_normalized_time[train_size+val_size:]

avai_matrix_test = vectorized_avai_index_df[train_size+val_size:]
nan_matrix_test = vectorized_nan_index_df[train_size+val_size:]
pad_matrix_test = vectorized_pad_index_df[train_size+val_size:]

In [51]:
#check number of avai instances in test set
print('Checking number of available instances in test set:...')
check_avai = avai_matrix_test.copy()
check_avai = check_avai.reshape(avai_matrix_test.shape[0]*avai_matrix_test.shape[1], avai_matrix_test.shape[2])
check_avai = check_avai[np.all(check_avai == 1, axis=1)]
print('Number of available row: {}'.format(check_avai.shape[0]))
print(check_avai.shape[0] == avai_instance)

print('\n')

#check number of nan instances in test set
print('Checking number of nan instances in test set:...')
check_nan = nan_matrix_test.copy()
check_nan = check_nan.reshape(nan_matrix_test.shape[0]*nan_matrix_test.shape[1], nan_matrix_test.shape[2])
check_nan = check_nan[np.any(check_nan != 0, axis=1)]
print('Number of nan row: {}'.format(check_nan.shape[0]))
print(check_nan.shape[0] == nan_instance)

Checking number of available instances in test set:...
Number of available row: 2721
True


Checking number of nan instances in test set:...
Number of nan row: 2879
True


In [52]:
preprocessed_data_name = os.path.join(args.input_dir, 'preprocessed_data_{}.pkl'.format(args.nan_pct))
with open(preprocessed_data_name, 'wb') as f:
    pickle.dump(min_max_storage, f, protocol=2)
    pickle.dump(complete_matrix_w_normalized_time_train, f, protocol=2)
    pickle.dump(missing_matrix_w_normalized_time_train, f, protocol=2)
    pickle.dump(avai_matrix_train, f, protocol=2)
    pickle.dump(nan_matrix_train, f, protocol=2)
    pickle.dump(complete_matrix_w_normalized_time_val, f, protocol=2)
    pickle.dump(missing_matrix_w_normalized_time_val, f, protocol=2)
    pickle.dump(avai_matrix_val, f, protocol=2)
    pickle.dump(nan_matrix_val, f, protocol=2)
    pickle.dump(pad_matrix_val, f, protocol=2)
    pickle.dump(complete_matrix_w_normalized_time_test, f, protocol=2)
    pickle.dump(missing_matrix_w_normalized_time_test, f, protocol=2)
    pickle.dump(avai_matrix_test, f, protocol=2)
    pickle.dump(nan_matrix_test, f, protocol=2)
    pickle.dump(pad_matrix_test, f, protocol=2)
    pickle.dump(cols_w_time, f, protocol=2)
    pickle.dump(cols_w_normalized_time, f, protocol=2)